In [6]:
import pandas as pd
import re
from pycountry import countries

# Step 1: Load the data
data = pd.read_csv("raw_eli_lilly_studies.csv")

# Step 2: Clean data - Remove whitespace and drop duplicates
data = data.applymap(lambda x: x.strip() if isinstance(x, str) else x)
data = data.drop_duplicates()

# Step 3: Split 'Locations' column using '|' as delimiter - long format
locations_split = data['Locations'].str.split('|', expand=True)
locations_long = locations_split.stack().reset_index(level=1, drop=True).rename('Location')
data = data.drop(columns=['Locations']).join(locations_long)

# Step 4: Extract Country (Fix Taiwan, Turkey, Czech Republic)
country_list = [country.name for country in countries]

def extract_country(location):
    location = location.lower()
    
    # Fix specific country name issues
    if "taiwan" in location:
        return "Taiwan"
    if "turkey" in location:
        return "Türkiye"  # Change to "Turkey" if preferred
    if "czech republic" in location:
        return "Czechia"

    # General country extraction
    for country in country_list:
        if country.lower() in location:
            return country
    return None

data['Location'] = data['Location'].astype(str)
data['Country'] = data['Location'].apply(extract_country)

# Step 5: Extract State (without City)
def extract_state(location):
    if pd.isna(location):
        return None
    
    # Remove special characters (like brackets)
    location_cleaned = re.sub(r"\[.*?\]|\(.*?\)", "", location).strip()
    
    # Split by comma
    parts = [part.strip() for part in location_cleaned.split(',')]
    
    # Identify country position
    country = extract_country(location)
    country_index = parts.index(country) if country in parts else len(parts)
    
    # Extract State (the part before the country)
    if country_index >= 2:
        return parts[country_index - 2]  # Extracting only the state
    return None

data['State'] = data['Location'].apply(extract_state)

# Step 6: Drop duplicates properly
data = data.drop_duplicates(subset=['NCT Number', 'Country', 'State'])

# Save the cleaned dataset
data.to_csv("cleaned_eli_lilly_studies.csv", index=False)


/var/folders/dz/d0n42s052ngg3fgsdssg1jcw0000gn/T/ipykernel_53825/2991093642.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: x.strip() if isinstance(x, str) else x)


Data cleaning completed! Taiwan, Turkey, and Czech Republic issues fixed.
